In [123]:
import fitz
import pathlib
import os

# Get current file path
__file__ = 'find_replace.ipynb'
current_path = pathlib.Path(__file__).parent.absolute()

# define input and outputpath
pdf_list = ['TEST PDF INPUT TEMPLATE.pdf', '3_giberish template.pdf', '4_giberish template.pdf','strings and tables.pdf']
input_file = os.path.join(current_path, 'input', pdf_list[3])
output_filename = 'result1.pdf'
output_file_path = os.path.join(current_path, 'output', output_filename)

REPLACEMENT_LIST = [
    #input for TEST PDF INPUT TEMPLATE.pdf
     {'{FULL NAME}': 'Prabahat','{number1}': '1234567890','{number2}': 'AA123456','{date11111}': '17/12/2021'},
    #input for 4_giberish template.pdf
     {'FULLNAME}': 'JLKSD', '{PASSPORT}': 'JFDSKLFHSDLK', '{GENDER}': 'Male', '{DATE3}': '20.09.2021', '{DATE1}': '20.09.2021','{DATE4}': '20.09.2020', '{DATE2}': '20.09.2021', '{DOB}': '14','{FILEN}': '10843', '{SOMENR}':'SOME','{ANR}':'ANR'},
    #input for 3_giberish template.pdf
     {'{str1}': 'XXX', '{str2}': 'xxx', '{str3}': 'xxx', '{str4}': 'xxx', '{str5}': 'xxx', '{DATE2}': '12.04.1197'},
    #input for string and tables.pdf
     {'{28698}': '289023', '{06092021}': '2692121', '{01SSS}': '01CCC','{23456890}': '23456890', '{759478229}': '759478221', '{74293047892374827427}': '94293047892374827420', '{02DD}': '02DD', '{47209748239084791749111}':'4720974823908479174', '{STR3438294}':'STR34', '{STR3553}': 'STR3'}

]

# This is just for example to pass images accroding to pdf
REPLACEMENT_DICT = REPLACEMENT_LIST[3] 

# DEFINE COLORS
white = (1, 1, 1)
black = (0, 0, 0)
blue = (0, 0, 0.6)

In [130]:
doc = fitz.open(input_file)
print(doc.metadata) 
#open page one by one
for page in doc:

    # get page info
    # print(page.get_texttrace())

    from fitz.fitz import TEXT_ALIGN_CENTER, TEXT_ALIGN_RIGHT
    # search for text in dict
    for rep_key, rep_value in REPLACEMENT_DICT.items():
        text_instances = page.searchFor(rep_key)
        for inst in text_instances:
        #   Get text color, font and size here
            text_info = page.get_text('dict', clip=inst)['blocks'][0]['lines'][0]['spans'][0]
            text_info_dict = {
                'fontsize': text_info.get('size',12),
                'border_width':10,
                'color':fitz.sRGB_to_pdf(text_info.get('color', black)),
                 'align': TEXT_ALIGN_RIGHT

            }

            rect = fitz.Rect(inst.x0-fitz.getTextlength(rep_value)/5, inst.y0+2, inst.x1, inst.y1+2)
            annot = page.addRedactAnnot(rect)
            # if you want to make sure to keep overlapping images:
            page.apply_redactions(images=fitz.PDF_REDACT_IMAGE_REMOVE)
            page.insert_textbox(rect, rep_value, **text_info_dict)
    
#   replace the images
#   grabic_3doc.pdf  replace_image_dict = {'Im1':'1.png', 'Im2':'3.png', 'Im3':'bar.png', 'Im4':'QR.jpg', 'Im0':'5.png'}
    replace_image_dict = {'Im1':'QR.jpg', 'Im2': 'mainqr.png'}
    ImageList = sorted(page.get_images(full=True))
    for img in ImageList:
        if img[7] in replace_image_dict:
            img_rect = page.getImageBbox(img)
            page.addRedactAnnot(rect, text=" ")
            page.apply_redactions(images=fitz.PDF_REDACT_IMAGE_REMOVE)
            replace_image_path = os.path.join(current_path, 'assets', replace_image_dict[img[7]])
            page.insertImage(img_rect, filename=replace_image_path)
        

{'format': 'PDF 1.7', 'title': 'Результаты исследований', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe Illustrator CC 23.1 (Macintosh)(Infix Pro)', 'producer': 'cairo 1.16.0 (https://cairographics.org)', 'creationDate': "D:20210702223419+03'00'", 'modDate': 'D:20211118163640', 'trapped': '', 'encryption': None}


In [128]:
print(page.rect)
#  change background image
background_img_filename = 'raw background.png'
background_img_path = os.path.join(current_path, 'assets', background_img_filename)

# insert background image to the full page
# full_img_rect = fitz.Rect(0,0,page.rect)
# page.add_redact_annot(full_img_rect)
# page.apply_redactions(images=fitz.PDF_REDACT_IMAGE_NONE)
page.insertImage(page.rect, filename=background_img_path, overlay=False, alpha=100, keep_proportion=False)


Rect(0.0, 0.0, 841.0, 595.0)


1142

In [129]:
# save doc
doc.save(output_file_path, garbage=4, deflate=True, clean=True)
print("completed>>>>>")


completed>>>>>
